In [62]:
#!pip install gradio langchain-google-genai langchain tensorflow pillow

In [63]:
import gradio as gr
import numpy as np
from PIL import Image
from dotenv import load_dotenv
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

In [64]:
# 3. Load environment variables
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = "gemini-2.0-flash"

In [65]:
# 4. Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model=GEMINI_MODEL,
    temperature=0.4
)

In [66]:
# 5. Load trained CNN model
cnn_model = load_model("saved_models/dementia_cnn_sequential_4_model_V2.keras")

In [67]:
# # 6. Class label map
# label_map = {
#     0: "Non-dementia",
#     1: "Very mild dementia",
#     2: "Mild dementia",
#     3: "Moderate dementia"
# }

In [68]:
label_map = ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented']

In [69]:
# 7. Function: Classify MRI image
def predict_from_mri(image):

    # 1. Convert to grayscale
    image = image.convert("L")

    # 2. Resize to match model input
    image = image.resize((224, 224))

    # 3. Convert to array
    img_array = img_to_array(image)  # shape: (224, 224, 1)

    # 4. Repeat the grayscale channel to create 3 channels
    img_array = np.repeat(img_array, 3, axis=-1)  # shape: (224, 224, 3)

    # 5. Normalize pixel values to [0, 1]
    img_array = img_array / 255.0

    # 6. Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)  # shape: (1, 224, 224, 3)

    # 7. Predict using the model
    preds = cnn_model.predict(img_array)
    label = label_map[np.argmax(preds)]
    
    return label


In [70]:
# 8. Function: Use Gemini to generate natural language explanation
def generate_gemini_summary(input_text):
    prompt = PromptTemplate.from_template("""
You are a dementia diagnostic assistant.

Interpret the following patient data or classification and explain what it means in simple, compassionate terms for a family or clinical audience:

"{input_text}"

Do not diagnose or suggest treatment. Just explain what it may indicate about dementia stages.
""")
    final_prompt = prompt.format(input_text=input_text)
    response = llm.invoke(final_prompt)
    return response.content.strip()

In [71]:
# 9. Combined chatbot function
def multimodal_chatbot(mri_image, symptom_text):
    responses = []

    if mri_image:
        diagnosis = predict_from_mri(mri_image)
        responses.append(f"**MRI Classification:** {diagnosis}")
        explanation = generate_gemini_summary(f"MRI indicates: {diagnosis}")
        responses.append(f"**Gemini Summary for MRI:**\n{explanation}")

    if symptom_text:
        explanation = generate_gemini_summary(f"Symptoms: {symptom_text}")
        responses.append(f"**Gemini Summary for Symptoms:**\n{explanation}")

    if not responses:
        return "Please upload an MRI image or enter symptom information."

    return "\n\n".join(responses)

In [72]:
# 10. Gradio UI
gr.Interface(
    fn=multimodal_chatbot,
    inputs=[
        gr.Image(type="pil", label="Upload MRI Image"),
        gr.Textbox(lines=4, placeholder="Describe symptoms here...", label="Symptom Description")
    ],
    outputs="markdown",
    title="Dementia MRI & Symptom Assistant (Powered by Gemini)",
    description="Upload an MRI or enter symptoms. The assistant will classify dementia and explain the findings using Google's Gemini 2.0 model."
).launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
